# Generate forecast code with OpenAI Responses API

This notebook sends a single prompt to several models, extracts R code blocks from responses, and saves them as .R files.


In [ ]:
import re
from pathlib import Path

from openai import OpenAI, BadRequestError
client = OpenAI()

In [ ]:
prompt_original = """
Design a method to forecast the number of people from Ukraine arriving in this country in the next 3 months. The only data you have is weekly totals of:
1. visa applications
2. visas issued
3. Type of visa ("Ukraine Family Scheme", "Ukraine Sponsorship Scheme", or "Government sponsored")
4. people with visas arriving in the country.
Use this data to engineer new features, if it improves the accuracy of the forecast.
Implement the forecast in R.
"""

prompt_with_data_caveat = """
Design a method to forecast the number of people from Ukraine arriving in this country in the next 3 months. The only data you have is weekly totals of:
1. visa applications
2. visas issued
3. Type of visa ("Ukraine Family Scheme", "Ukraine Sponsorship Scheme", or "Government sponsored")
4. people with visas arriving in the country.
There is 22 weeks of historical data available in total.
Use this data to engineer new features, if it improves the accuracy of the forecast.
Implement the forecast in R.
"""

prompt_simulation = """
Code a numerical simulation to predict the number of people from Ukraine arriving in this country in the next 3 months. The only data you have is weekly totals of:
1. visa applications
2. visas issued
3. Type of visa ("Ukraine Family Scheme", "Ukraine Sponsorship Scheme", or "Government sponsored")
4. people with visas arriving in the country.
Use this data to engineer new features, if it will improve the simulation.
Implement the simulation in R.
"""

# https://platform.openai.com/docs/models
models = [
    "gpt-5.2",
    "gpt-4.1",
    "gpt-4o",
    "gpt-4",
    "o3",
    "o1",
    "gpt-3.5-turbo"
]

Function to extract blocks of R code from OpenAI responses.

In [ ]:
def extract_code_blocks(text):
    pattern = r"```[rR]?\s*([\s\S]*?)```"
    return [m.strip() for m in re.findall(pattern, text, flags=re.MULTILINE)]

### Generate forecast code for each model

Choose which prompt to use and which output directory to save the files in.

In [ ]:
prompt = prompt_simulation

output_dir = Path("../simulations")

Run each model with the same prompt and save the R code to files.

In [ ]:
for model in models:
    print(f"Requesting model: {model}")

    # Params to use for calls to all models
    base = {"model": model, "input": prompt}

    # Start by trying to call the model as if it can reason
    try:
        response = client.responses.create(**base, reasoning={"effort": "high"})
    # Some of them can't, so retry without the `reasoning` parameter
    except BadRequestError as e:
        # Only retry on "reasoning param not supported" style errors
        msg = str(e).lower()
        if "reasoning" in msg and ("unsupported" in msg or "not supported" in msg or "unknown parameter" in msg):
            response = client.responses.create(**base)
        else:
            raise

    # print(response.output_text)

    # Try to extract and save blocks of R code
    blocks = extract_code_blocks(response.output_text)

    if not blocks:
        print(f"Warning: no R code blocks found for {model}")
        code = response.output_text
    else:
        print(f"Saving R code for {model}")
        code = "\n\n".join(blocks)

    code_file = output_dir / f"{model}.R"
    code_file.write_text(code, encoding="utf-8")

    # Also save the entire response, just in case
    response_file = output_dir / f"{model}.md"
    response_file.write_text(response.output_text, encoding="utf-8")

    print(f"Finished with {model}")
